In [1]:
from ultralytics import YOLO
import os
import shutil
import pandas as pd
import time
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import f1_score as f1
from sklearn.metrics import precision_score as precision
from sklearn.metrics import recall_score as recall
from sklearn.metrics import accuracy_score as accuracy

pd.set_option('display.max_rows', 300)

In [2]:
classes = [0, 1, 2, 3]
answers = [0, 1, 2, 3]

model = YOLO('/Users/makoto/dev/YOLO/training_log/l_22/weights/best.pt')

image_dir = '/Users/makoto/dev/YOLO/original_data/images/'
label_dir = '/Users/makoto/dev/YOLO/original_data/labels/'
output_dir = '/Users/makoto/Desktop/output/'

conf = 0.1

In [3]:
class PredictAndCreateDf:
    
    def __init__(self, model, image_dir, label_dir, output_dir, conf):
        
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.output_dir = output_dir
        self.model = model
        self.conf = conf
    
    def _predict(self, image_path):
        
        result = self.model.predict(image_path, project=self.output_dir, save=True, imgsz=640, conf=self.conf, exist_ok=True)
        
        result_object = result[0]
        pred = result_object.boxes.cls.tolist()
        pred = [int(c) for c in pred]
        
        return pred
    
    def _answer(self, image_path):
        
        answer =[]
        
        image_name = image_path.split('/')[-1]
        file_head = image_name.split('.')[0]
        file_name = f'{file_head}.txt'
        file_path = f'{self.label_dir}/{file_name}'
    
        with open(file_path, 'r') as f:
            label_lines = f.readlines()
            
        for label_line in label_lines:
            numbers = label_line.split()
            number = int(numbers[0])
            answer.append(number)
        
        return answer  
        
    def _create_row(self, image_path):
        
        classes = [0, 1, 2, 3]
        answers = [0, 1, 2, 3]
        
        pred = self._predict(image_path)
        answer = self._answer(image_path)

        row = [0]*9
    
        row[0] = image_path.split('/')[-1]
    
        for i, class_id in enumerate(classes):
            row[i+1] = 1 if class_id in pred else 0
        
        for i, class_id in enumerate(answers):
            row[i+5] = 1 if class_id in answer else 0
        
        return row
    
    def create_df(self):
        
        df = pd.DataFrame(columns=['image_number', 'predict_0', 'predict_1', 'predict_2', 'predict_3', 'answer_0', 'answer_1', 'answer_2', 'answer_3'])
        
        image_paths = []
        for file in os.listdir(self.image_dir):
            image_path = os.path.join(self.image_dir, file)
            if os.path.isfile(image_path):
                image_paths.append(image_path)
        
        for image_path in image_paths:
            row = self._create_row(image_path)
            row = pd.Series(row, index=df.columns)
            df = pd.concat([df, row.to_frame().T], ignore_index=True)
            
        return df

In [4]:
def get_metrics(df):
    
    list_cm =[]
    list_f1 = []
    list_precision =[]
    list_recall =[]
    list_accuracy =[]
    
    for class_id in classes:
        
        pred = df[f'predict_{class_id}']
        answer = df[f'answer_{class_id}']
        
        globals()[f'cm_{class_id}'] = cm(answer, pred)
        list_cm.append(globals()[f'cm_{class_id}'].flatten())
        
        globals()[f'f1_{class_id}'] = f1(y_true=answer, y_pred=pred)
        list_f1.append(globals()[f'f1_{class_id}'])
        
        globals()[f'precision_{class_id}'] = precision(answer, pred)
        list_precision.append(globals()[f'precision_{class_id}'])
        
        globals()[f'recall_{class_id}'] = recall(answer, pred)
        list_recall.append(globals()[f'recall_{class_id}'])
        
        globals()[f'accuracy_{class_id}'] = accuracy(answer, pred)
        list_accuracy.append(globals()[f'accuracy_{class_id}'])
        
    return list_cm, list_f1, list_precision, list_recall, list_accuracy

In [5]:
def main():
    
    start = time.time()
    
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.mkdir(output_dir)
    
    predict_and_create_df = PredictAndCreateDf(model, image_dir, label_dir, output_dir, conf)
    
    df = predict_and_create_df.create_df()
    
    df[['predict_0', 'predict_1', 'predict_2', 'predict_3', 'answer_0', 'answer_1', 'answer_2', 'answer_3']] = df[['predict_0', 'predict_1', 'predict_2', 'predict_3', 'answer_0', 'answer_1', 'answer_2', 'answer_3']].astype(int)
    
    cm = get_metrics(df)[0]
    f1 = get_metrics(df)[1]
    precision = get_metrics(df)[2]
    recall = get_metrics(df)[3]
    accuracy = get_metrics(df)[4]
    
    cm_all = [0] * len(cm[0])
    
    for sublist in cm:
        for i, element in enumerate(sublist):
            cm_all[i] += element
    
    tn, fp, fn, tp = cm_all
    
    accuracy_all = (tn + tp) / (tn + fp + fn + tp)
    precision_all = tp / (tp + fp) 
    recall_all = tp / (tp + fn)
    f1_all = (2 * precision_all * recall_all) / (precision_all + recall_all)
    
    print('All Items')
    print('TN  FP\nFN  TP')
    print(f'{tn}  {fp}\n{fn}  {tp}')
    print(f'F1 : {f1_all}')
    print(f'Precision : {precision_all}')
    print(f'Recall : {recall_all}')
    print(f'Accuracy : {accuracy_all}')
    print('----------------------------------------------------------------')
    
    for class_id in classes:
        tn, fp, fn, tp = cm[class_id]
        print(f'Class{class_id}')
        print('TN  FP\nFN  TP')
        print(f'{tn}  {fp}\n{fn}  {tp}')
        print(f'F1 : {f1[class_id]}')
        print(f'Precision : {precision[class_id]}')
        print(f'Recall : {recall[class_id]}')
        print(f'Accuracy : {accuracy[class_id]}')
        print('----------------------------------------------------------------')
    
    end = time.time() 
    
    return end - start

In [6]:

process_time = main()
print(process_time)


image 1/1 /Users/makoto/dev/YOLO/original_data/images/book00119.JPG: 640x480 1 KodaiChugokuNo24Jikan, 1 ChainSawMan12, 1 ChugokugoNavi202204, 291.4ms
Speed: 2.6ms preprocess, 291.4ms inference, 560.6ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /Users/makoto/Desktop/output/predict

image 1/1 /Users/makoto/dev/YOLO/original_data/images/book00125.JPG: 640x480 1 KodaiChugokuNo24Jikan, 299.8ms
Speed: 1.8ms preprocess, 299.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /Users/makoto/Desktop/output/predict

image 1/1 /Users/makoto/dev/YOLO/original_data/images/book00131.JPG: 640x480 (no detections), 407.8ms
Speed: 2.6ms preprocess, 407.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /Users/makoto/Desktop/output/predict

image 1/1 /Users/makoto/dev/YOLO/original_data/images/book00079.JPG: 640x480 1 UtukushiNihongoErabiJiten, 1 KodaiChugokuNo24Jikan, 1 ChainSawMan12, 300.9ms
Speed: 2.2ms preproc

In [7]:
process_time / 233

0.3920650164968466